In [1]:
import numpy as np
from datetime import date
import pandas as pd
import os
from datetime import datetime, timedelta
%matplotlib inline
import nltk
from statistics import mean 
from dotenv import load_dotenv
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import json
import requests
load_dotenv()
import os
from nltk.sentiment.vader import SentimentIntensityAnalyzer

%matplotlib inline

In [2]:
analyzer = SentimentIntensityAnalyzer()

In [3]:
features_df = pd.read_csv("features_df.csv") 

In [4]:
features_df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
features_df=features_df.set_index('Date')

In [5]:
#features_df = pd.Timestamp('Date').to_datetime()
features_df.index = pd.to_datetime(features_df.index).date


In [6]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/aljohara/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
dates=pd.DataFrame({'Date':pd.Series(features_df.index)})
dates.head()

,Date
0,2018-07-03
1,2018-07-05
2,2018-07-06
3,2018-07-09
4,2018-07-10


# Fetch News Articles
### - Gnews
### - Stock News API

In [8]:
articles_df = pd.DataFrame(columns=['date','text','source', 'compound'])

In [9]:
# Read your api key environment variable
api_key = os.getenv("gnews")


In [10]:
#GNEWS:
import requests
import json


request_url = "https://gnews.io/api/v3/search"
query = 'tesla'

import time
n = len(dates)
# we go over all dates in stock price df, we jump 5 days to save more iterations (API limit), fetching a week before the current date
for i in range(0,n,5):
    
    date = dates.loc[i]['Date']
    # minus 1 week
    date_begin = date - pd.to_timedelta(1, unit='w')
    #print(date,date_begin)

    # change date to string
    date = str(date)
    date_begin = str(date_begin)
    
    # Create parameterized url

    params = {"q":query,'token':api_key,'max':'100','mindate':date_begin,'maxdate':date}
    
    # Submit request and format output as json
    response_data = requests.get(request_url,params=params).json()
    
    # To stop when we reach API limit
    try:
        count_articles = response_data['articleCount']
        returned_articles_gapi = response_data['articles']
    except:
        print('error', response_data)
        break
    
    # add to articles_df
    for article in returned_articles_gapi:
        date = article['publishedAt']
        date = pd.to_datetime(date).date()
        text = article['title'] + ' '
        text += article['description']
        articles_df  = articles_df.append({'date':date,'text':text, 'source':'gapi'},ignore_index=True)
    # Sleep for 5 seconds because API blocks sending too many requests in short time 
    time.sleep(5)

In [11]:
# Stock News API
stocknewsapi_key = os.getenv("stock_news_api")

In [12]:
request_url = "https://stocknewsapi.com/api/v1"
query = 'TSLA'

n = len(dates)
for i in range(n):
    date = dates.loc[i]['Date']
    # minus 1 week
    date_begin = date - pd.to_timedelta(1, unit='w')
    #date_begin = date_begin.date()


    # change it to MMDDYYYY formate for stocknewsapi call
    date = str(date)
    date = date.split('-')
    date = date[1]+date[2]+date[0]
    date_begin = str(date_begin)
    date_begin = date_begin.split('-')
    date_begin = date_begin[1]+date_begin[2]+date_begin[0]
    
    date = date_begin+'-'+date

    # Create parameterized url
    params = {"tickers":query,'token':stocknewsapi_key,'items':'50','fallback':'true','date':date}
    
    # Submit request and format output as json
    response_data = requests.get(request_url,params=params).json()
    
    articles = response_data['data']

     # add to articles_df
    for article in articles:
        date = article['date']
        date = pd.to_datetime(date).date()
        text = article['title'] + ' '
        text += article['text']
        sentiment = article['sentiment']
        articles_df  = articles_df.append({'date':date,'text':text, 'source':'stocknews'},ignore_index=True)
        

## Sentiment Analysis of Gnews & Stock News API


In [13]:
# Sentiment Analysis 
n = len(articles_df)
for i in range(n):
    article = articles_df.iloc[i]
    text = article['text']
    sentiment = analyzer.polarity_scores(text)    
    articles_df.loc[i]['compound'] = sentiment['compound']

In [14]:
articles_df

,date,text,source,compound
0,2018-07-02,Construction workers offer tips to stay cool T...,gapi,0.3818
1,2018-07-01,NYT ed board: Time for new Dem leadership TRAD...,gapi,-0.4019
2,2018-06-30,"Elon Musk stole my dad’s farting unicorn art, ...",gapi,0.7096
3,2018-06-29,Evergreen artist wants Elon Musk to puff up pa...,gapi,-0.296
4,2018-06-29,Virgin Orbit Gears Up for Captive Carry Test F...,gapi,0.5423
...,...,...,...,...
25943,2020-06-23,"Apple, Google, Tesla CEOs slam Trump’s move to...",stocknews,-0.7351
25944,2020-06-23,"Google, Apple, Tesla CEOs All Blast Trump Fore...",stocknews,-0.2263
25945,2020-06-23,What Pandemic? These 5 Large Caps Are Up More ...,stocknews,0
25946,2020-06-23,Tesla's Third-Row Model Y May Come Early The c...,stocknews,0.7845


### Calculating Mean Compound and Adding Other Machine Learning Features

In [15]:
import datetime
new_features_df = features_df.copy()
new_features_df.drop(columns=['daily_returns_tlt','daily_returns_tlt_volume','daily_returns_tsla','daily_returns_tsla_volume'],inplace=True)
new_features_df.index = pd.to_datetime(new_features_df.index)

new_features_df = features_df.shift(1)
new_features_df['target'] = features_df['target']


i = 0
for date in new_features_df.index:
    start_date = date - pd.to_timedelta(3, unit='d')
    
    mask = (articles_df['date'] > start_date) & (articles_df['date'] <= date)
   
    count_articles = articles_df.loc[mask].count()
    means = articles_df.loc[mask].mean()

    
    new_features_df.loc[date,'mean_compound'] = means['compound']
    
    new_features_df.loc[date,'daily_returns_tsla'] = features_df.loc[date,'daily_returns_tsla']

    i += 1

In [16]:
new_features_df.head()

,daily_returns_tlt,daily_returns_tlt_volume,daily_returns_tsla,daily_returns_tsla_volume,target,mean_compound
2018-07-03,NaN,NaN,-0.072060,NaN,0,-0.016740
2018-07-05,0.005109,-0.625701,-0.005836,-0.356887,0,-0.450450
2018-07-06,0.002869,0.044150,-0.000647,0.456280,0,-0.423822
2018-07-09,0.003106,0.199468,0.030365,-0.492655,1,0.122037
2018-07-10,-0.006112,0.201659,0.012818,-0.190807,1,0.097726


In [17]:
new_features_df.rename(columns={"mean_compound": "gnews_&_stockapi_mean_compound"}, inplace=True)

In [18]:
# save results to CSV file 
new_features_df.to_csv('allapinewscompound.csv')